<a href="https://colab.research.google.com/github/SinaQP/Tempus/blob/master/Tempus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain langchain-community langchain-openai jdatetime --upgrade --quiet

ERROR: Operation cancelled by user


In [ ]:
%pip install jdatetime

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
              

In [ ]:
from langchain.chat_models import ChatOpenAI

AVALAI_API_KEY=""
AVALAI_BASE_URL="https://api.avalai.ir/v1"
chat = ChatOpenAI(model="gpt-4o-mini", openai_api_key=AVALAI_API_KEY, base_url=AVALAI_BASE_URL, temperature=0.2)

In [ ]:
result = chat.invoke("Hello").content
result

'Hello! How can I assist you today?'

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import date

# Define the model for a single activity
class Activity(BaseModel):
    date: str = Field(..., description="تاریخ فعالیت به صورت YYYY/MM/DD (جلالی)")
    start_time: str = Field(default="08:00", description="ساعت شروع فعالیت")
    end_time: str = Field(default="15:00", description="ساعت پایان فعالیت")
    project: Optional[str] = Field(default=None, description="نام پروژه")
    activity_subject: str = Field(..., description="موضوع فعالیت")
    activity_summary: Optional[str] = Field(default=None, description="خلاصه‌ای از فعالیت")

# Define the model for the entire response
class ActivitiesResponse(BaseModel):
    activities: List[Activity]


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
import jdatetime
from langchain_core.output_parsers import PydanticOutputParser

current_jalali_datetime = str(jdatetime.datetime.now().date()).replace("-", '/')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
parser = PydanticOutputParser(pydantic_object=ActivitiesResponse)

template = """
شما یک دستیار هوشمند هستید که اطلاعات مربوط به فعالیت‌های روزانه را استخراج می‌کنید.
اطلاعاتی که باید استخراج شوند:
- **date**: تاریخ فعالیت (اگر تاریخ ذکر نشده بود، از تاریخ امروز استفاده کن: {current_jalali_datetime})
- **start_time**: ساعت شروع فعالیت (اگر ذکر نشده بود، پیش‌فرض 08:00 است)
- **end_time**: ساعت پایان فعالیت (اگر ذکر نشده بود، پیش‌فرض 15:00 است)
- **project**: نام پروژه (اگر ذکر نشده بود، `null` برگردان)
- **activity_subject**: موضوع فعالیت (مانند جلسه، کدنویسی، پشتیبانی، آموزش)
- **activity_summary**: خلاصه‌ای از فعالیت (اگر ذکر نشده بود، یک خلاصه بسیار ساده بر اساس موضوع فعالیت ایجاد کن و یکم بزرگترش کن)

پاسخ را به صورت JSON برگردان. به عنوان مثال:
```json
{{
    "activities": [
        {{
            "date": "YYYY/MM/DD",
            "start_time": "08:00",
            "end_time": "11:00",
            "project": "رهتال",
            "activity_subject": "کدنویسی",
            "activity_summary": "اضافه کردن api های مورد نیاز فرانت اند"
        }}
    ]
}}
متن ورودی: "{query}"
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])
query = "کدنویسی سامانه رهبری دیجیتال از 9 تا 12 در 24 دی"
messages = chat_prompt.invoke({
"query": query,
"format_instructions": parser.get_format_instructions(),
"current_jalali_datetime": current_jalali_datetime
})
result = chat.invoke(messages)

In [ ]:
try:
    joke_object = parser.parse(result.content)
    print(joke_object.activities[0])
    print(joke_object.activities[0].activity_summary)
    print(joke_object.activities[0].project)
except Exception as e:
    print(e)

date='1403/10/24' start_time='09:00' end_time='12:00' project='رهبری دیجیتال' activity_subject='کدنویسی' activity_summary='توسعه و کدنویسی سامانه رهبری دیجیتال'
توسعه و کدنویسی سامانه رهبری دیجیتال
رهبری دیجیتال
